# Examples

## Simple function minimization

Let's minimize the following simple function:

$$
f(x, y) = (x - 3.6)^2 - 2 (y+1.1)^2
$$

The global minimum is obvious: $f=0$ for $x=3.6, y=-1.1$. We now want to find this minimum using `iwopy`, in a setup as follows:

- Parameter bounds: $x, y \in [-5,5]$
- Unconstrained minimization
- Gradient based optimizer [IPOPT](https://esa.github.io/pagmo2/docs/cpp/algorithms/ipopt.html) from [pygmo](https://esa.github.io/pygmo2/)
- Analytical gradients

The usage of finite difference gradients will be demonstrated directly after. We start by importing the required classes from the `iwopy` package:

In [1]:
from iwopy import SimpleProblem, SimpleObjective
from iwopy.interfaces.pygmo import Optimizer_pygmo

The `SimpleProblem` will be instantiated with two float type variables `x` and `y` which will be passed on to all linked objectives and constraints. The `SimpleObjective` (and also the `SimpleConstraint` classes assume the same variables as the problem, in the same order. We can therefore implement the above function $f(x, y)$ and the derivative 
$$
g(v, x, y) = \left\{ 
\begin{array}{ll}
\mathrm df / \mathrm dx \ ,& \text{if} \ v = 0 \\
\mathrm df / \mathrm dy \ ,& \text{if} \ v = 1
\end{array}
\right.
$$ 
in a straight forward manner:

In [2]:
class MinFunc(SimpleObjective):
    
    def f(self, x, y):
        return (x - 3.6)**2 + 2*(y + 1.1)**2
    
    def g(self, v, x, y, components):
        return 2*(x - 3.6) if v == 0 else 4*(y + 1.1)

Notice that the `components` argument of the function `g` is not used here since `f` is a single-component function.

Otherwise, the parameter `n_components` has to be passed to the `__init__` function of `SimpleObjective`, and `f` has to return a list of scalars. In that case also `g` has to return a list of results, with same length as the requested `components`. The same rules apply to classes derived from `SimpleConstraint`.

We can now proceed and setup the problem:

In [3]:
problem = SimpleProblem(
    name="minf", 
    float_vars=["x", "y"],
    init_values_float=[0.0, 0.0],
    min_values_float=[-5.0, -5.0],
    max_values_float=[5.0, 5.0],
)
problem.add_objective(MinFunc(problem))
problem.initialize()

Problem 'minf' (SimpleProblem): Initializing
--------------------------------------------
  n_vars_int   : 0
  n_vars_float : 2
--------------------------------------------
  n_objectives : 1
  n_obj_cmptns : 1
--------------------------------------------
  n_constraints: 0
  n_con_cmptns : 0
--------------------------------------------


Note that in a similar way you can easily add constraints to the problem, by defining them in a class that is derived from `iwopy.Constraint` or `iwopy.SimpleConstraint` and then adding them via `problem.add_constraint(...)`. 

Adding additional objectives works in the same way, simply repeat `problem.add_objective(...)` as often as you want. However, be aware that not all optimizers can handle multiple objective cases.

Next, we create and initialize the solver:

In [4]:
solver = Optimizer_pygmo(
    problem,
    problem_pars=dict(c_tol=1e-3),
    algo_pars=dict(type="ipopt", tol=1e-4),
)
solver.initialize()
solver.print_info()


Algorithm name: Ipopt: Interior Point Optimization [deterministic]
	C++ class name: pagmo::ipopt

	Thread safety: none

Extra info:
	Last optimisation return code: Solve_Succeeded (value = 0)
	Verbosity: 1
	Individual selection policy: best
	Individual replacement policy: best
	Numeric options: {tol : 0.0001}



Ok, we are now ready - let's solve the problem!

In [5]:
results = solver.solve()
solver.finalize(results)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************


 objevals:        objval:      violated:    viol. norm:
         1          15.38              0              0
         2        9.23375              0              0
         3        9.23375              0              0
         4        9.23375              0              0
         5        9.23375              0              0
         6        9.23375              0              0
         7        1.60594              0              0
         8      0.0440281              0              0
         9     0.00157443              0              0
        10    1.35988e-06              0           

In [6]:
print("x,y =", results.vars_float)
print("f =", results.objs[0])

x,y = [ 3.59999901 -1.10000039]
f = 1.2852212531163044e-12


Next, we want to explore finite difference gradient calculations for the same example. We can either remove the function `g(v, x, y, components)` from the `MinFunc` class, or create it invoking the parameter `has_ana_derivs=False`. This will then ignore the analytical gradient definition in the class.

Additionally, we need to wrap the problem into the `DiscretizeRegGrid` problem wrapper and set the grid step size and the order of finite difference derivative calculations. This is the code for doing so:

In [7]:
from iwopy import DiscretizeRegGrid

In [11]:
problem = SimpleProblem(
    name="minf", 
    float_vars=["x", "y"],
    init_values_float=[0.0, 0.0],
    min_values_float=[-5.0, -5.0],
    max_values_float=[5.0, 5.0],
)
problem.add_objective(MinFunc(problem, has_ana_derivs=False))

In [17]:
gproblem = DiscretizeRegGrid(problem, deltas={"x": 0.01, "y": 0.01}, fd_order=2)
gproblem.initialize()

The `gproblem` object is now the `problem` on a grid, defined by the variable bounds and the specified `deltas`. We can use the same gradient based solver as above for solving this problem:

In [18]:
solver = Optimizer_pygmo(
    gproblem,
    problem_pars=dict(c_tol=1e-3),
    algo_pars=dict(type="ipopt", tol=1e-4),
)
solver.initialize()

results = solver.solve()
solver.finalize(results)


Optimisation return status: Solve_Succeeded (value = 0)

 objevals:        objval:      violated:    viol. norm:
         1          15.38              0              0
         2         9.2338              0              0
         3         9.2338              0              0
         4         9.2338              0              0
         5         9.2338              0              0
         6         9.2338              0              0
         7          1.606              0              0
         8      0.0440738              0              0
         9     0.00159727              0              0
        10    1.27606e-05              0              0
        11    1.77386e-08              0              0

Problem name: minf_grid
	C++ class name: pybind11::object

	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5, -5]
	Upper bounds: 

In [19]:
print("x,y =", results.vars_float)
print("f =", results.objs[0])

x,y = [ 3.59999901 -1.10000039]
f = 1.2852212529252677e-12


## Electrostatic potential minimization

The main purpose of `iwopy` is to be helpful when attacking more complicated optimization tasks than the minimization of simple analytical functions as above. As an example, we consider the minimization of an inverse distance type potential for a fully coupled system of $N$ particles in two dimensions. We can imagine such a system to be composed of $N$ individual point charges, each of them carrying the same electric unit charge.